In [1]:
import pandas as pd
import numpy as np
import os

from glob import glob
import yaml
import re

# custom package(s)
from utils import utils, synapse_login

syn = synapse_login.main()


UPGRADE AVAILABLE

A more recent version of the Synapse Client (3.0.0) is available. Your version (2.7.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 3.0.0 release notes

https://python-docs.synapse.org/build/html/news.html



Welcome, Nicholas Lee!



In [2]:
def get_dd(synId):
    df = syn.get("syn29860146")
    df = pd.read_csv(df.path, encoding="iso8859").fillna("")
    return df

In [3]:
def create_dict(list):
    if np.all(pd.isna(list)):
        return np.nan
    elif len(list) <= 1:
        return list
    else:
        # combine dictionaries
        new_dict = {}

        for d in list:
            for x in list:
                x_split = x.split("=")
                new_dict.update({x_split[0].strip(): x_split[1].strip()})
        return new_dict

In [4]:
md_df = syn.tableQuery(
    "SELECT * FROM syn51228435 WHERE metadataType = 'data dictionary'"
)
md_df = md_df.asDataFrame()
md_df

,id,study,name,resourceType,metadataType,dataType,assay
1_0,syn26559856,MRGWAS,MRGWAS_MendelianRandomizationAnalysis_DataDict...,metadata,data dictionary,NaN,[]
2_0,syn27373432,Aging-PheWAS,AgingPheWAS_GWASfiles_dataDictionary.txt,metadata,data dictionary,NaN,[]
3_0,syn27373454,Aging-PheWAS,AgingPheWAS_MetaXcan_dataDictionary.txt,metadata,data dictionary,NaN,[]
4_0,syn28526339,ADAMTS7,ADAMTS7_HABC_pheno_data_dictionary.txt,metadata,data dictionary,NaN,[]
5_0,syn28526340,ADAMTS7,ADAMTS7_MrOS_pheno_data_dictionary.txt,metadata,data dictionary,NaN,[]
6_0,syn28526343,ADAMTS7,ADAMTS7_SOF_pheno_data_dictionary.txt,metadata,data dictionary,NaN,[]
10_0,syn29860146,LLFS,LLFS-phenotype-DD.04272022.csv,metadata,data dictionary,NaN,[snpArray]
11_0,syn29860148,LLFS,LLFS-pedigree-DD.04272022.csv,metadata,data dictionary,NaN,[snpArray]


In [5]:
combined_df = pd.DataFrame()

for i, v in md_df.iterrows():
    temp_df = get_dd(v["id"])
    temp_df["parent"] = v["name"]
    combined_df = pd.concat([combined_df, temp_df])

combined_df = combined_df.drop_duplicates(subset=["domain", "VARNAME"])
combined_df = combined_df.sort_values(by=["domain", "VARNAME"])
combined_df.reset_index(drop=True, inplace=True)
# get value columns
r = re.compile("VALUES|Unnamed*")
combined_cols = list(filter(r.match, combined_df.columns.tolist()))
combined_df["VALUES"] = combined_df[combined_cols].values.tolist()
combined_cols = [x for x in combined_cols if x != "VALUES"]
combined_df = combined_df.drop(columns=combined_cols)
del combined_cols

In [6]:
combined_df["VALUES"] = combined_df["VALUES"].apply(
    lambda x: [y for y in x if len(y) > 0]
)

In [7]:
combined_df["VALUES"] = combined_df["VALUES"].apply(
    lambda x: np.nan if len(x) == 0 else x
)
combined_df["VALUES"] = combined_df["VALUES"].apply(create_dict)
combined_df = combined_df.fillna("")
combined_df = combined_df.set_index("VARNAME")
combined_df = combined_df.drop(columns=["parent"])
with open("../_data/combined_df.json", "w") as f:
    f.write(combined_df.to_json(default_handler=str, orient="index"))

In [12]:
combined_df

,domain,VARDESC,COLLINTERVAL,UNITS,COMMENT1,MIN,MAX,VALUES
VARNAME,,,,,,,,
consent,,Consent Group,,,,1.0,3.0,"{'1': 'General Research Use (IRB) (GRU-IRB)', ..."
country,,US or Denmark field center,,,,,,"{'US': 'US', 'DK': 'Denmark'}"
subject,,Deidentified Subject ID,,,,,,
visitcode,,Visit Code,,,,1.0,4.0,"{'1': 'Visit 1', '3': 'Visit 2 Returning Parti..."
_Height,Bone Health,Average Stand Height in cm,Collected in Visit 1 & Visit 2,cm,Domain: Bone Health,128.5,205.2,
...,...,...,...,...,...,...,...,...
hyst,Reproductive,Ever had a hysterectomy (surgery to remove ute...,Collected in Visit 1 & Visit 2,,Domain: Reproductive,0.0,1.0,"{'.D': 'Don't Know', '.R': 'Refused', '.U': 'U..."
menopage,Reproductive,Age of menopause was reached,Collected in Visit 1 & Visit 2,years,Domain: Reproductive,25.0,72.0,
nbirth,Reproductive,How many pregnancies resulted in a live child?,Collected in Visit 1 & Visit 2,,Domain: Reproductive,0.0,16.0,"{'.D': 'Don't Know', '.R': 'Refused'}"


# Grant Information


In [ ]:
# Pull grant information
grants = syn.tableQuery(
    "SELECT * FROM syn51209786").asDataFrame()["grantNumber"]

grantIds = []
for i in grants:
    grantIds += i

grantIds = ",".join([i.strip() for i in grantIds])

dm.loc[
    dm["Attribute"].str.contains("grant", flags=re.IGNORECASE), "Valid Values"
] = grantIds